In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from keras import callbacks
from keras import models
from keras import layers
from keras.datasets import mnist
import matplotlib.pyplot as plt

#Step 1: Load and Split train and test datasets
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Print Sample train images
plt.figure(figsize=(5, 5))
plt.title("Training Images")
for i in range(9):
    plt.subplot(330+1+i)
    plt.imshow(train_images[i])
plt.show()


In [16]:
# Step 2: Preprocessing the image data
# Scale down the images to the [0, 1]
train_images = train_images.astype("float32") / 255
test_images = test_images.astype("float32") / 255

# Set image shape (28, 28, 1)
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

# Converts a class vector (integers) to binary class matrix.
train_labels = to_categorical(train_labels)
test_labels  = to_categorical(test_labels)

In [17]:
#Step 3: Build a network with multiple layers
num_classes = 10
input_shape = (28, 28, 1)
# Get Sequential to groups a linear stack of layers into a tf.keras.Model.
model = models.Sequential(
    [
        models.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
# Print model summary
model.summary()

In [18]:
# Step 4: Netowrk Compilation
# select an optimizer, a loss function & metrics for evaluation of model performance.
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics = ['accuracy'])

In [19]:
# Step 5: Train the model
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)
epochs = 20
batch_size = 128

# Fit the model to its training data
history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size,validation_data =(train_images, train_labels), 
                    callbacks =[earlystopping])

In [20]:
# Assess the training and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [21]:
# Assess the training and validation accuracy
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
plt.plot(epochs, accuracy, 'y', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [22]:
# Step 6: Evaluating the network’s performance
# Calculate Test loss and Test Accuracy
test_loss, test_acc = model.evaluate(test_images, test_labels)

# Print Test loss and Test Accuracy
print(f"Test Loss: {test_loss}\nTest Accuracy : {test_acc * 100} %")